# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [91]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
from pathlib import Path
from pprint import pprint

# Import API key
from api_keys import geoapify_key

In [128]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mareeba,-17.0000,145.4333,25.40,95,8,1.97,AU,1680081316
1,1,leningradskiy,69.3833,178.4167,-12.38,80,85,3.33,RU,1680081316
2,2,barrow,71.2906,-156.7887,-10.99,79,100,7.20,US,1680081316
3,3,sayansk,54.1088,102.1648,9.93,71,97,2.08,RU,1680081317
4,4,victoria point,-27.5833,153.3000,25.06,88,100,3.52,AU,1680081317


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [162]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 680,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.6
)    

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [149]:
# Narrow down cities that fit criteria and drop any results with null values
Max_temp = 24
Min_temp = 15
Wind_Speed = 4
Cloudiness = 5

city_data_reduced_df = city_data_df.loc[(city_data_df["Max Temp"] <= Max_temp) & (city_data_df["Max Temp"] >= Min_temp)]
city_data_reduced_df = city_data_reduced_df.loc[(city_data_reduced_df["Wind Speed"] <= Wind_Speed)]
city_data_reduced_df = city_data_reduced_df.loc[(city_data_reduced_df["Cloudiness"] <= Cloudiness)]

# Display sample data
city_data_reduced_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Step 3: Create a new DataFrame called `hotel_df`.

In [155]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
City = city_data_reduced_df["City"]
Country = city_data_reduced_df["Country"]
Lat = city_data_reduced_df["Lat"]
Lng = city_data_reduced_df["Lng"]
Humidity = city_data_reduced_df["Humidity"]
hotel_df = pd.DataFrame({
                "City" : City,
                "Country" : Country,
                "Lat" : Lat,
                "Lng" : Lng,
                "Humidity" : Humidity
                    })


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data

hotel_df.dtypes

City           object
Country        object
Lat           float64
Lng           float64
Humidity        int64
Hotel Name     object
dtype: object

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [156]:
# # Set parameters to search for a hotel
radius = 10000
limit = 20

params = {
        "categories": "accommodation.hotel", 
        "apiKey":geoapify_key,
        "limit":limit
        }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
 
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pisco - nearest hotel: La Portada
coahuayana - nearest hotel: No hotel found
chuy - nearest hotel: Alerces
verrettes - nearest hotel: Kay Haiti
birin - nearest hotel: Sun Hotel and Lodge
cidreira - nearest hotel: Hotel Castelo
pedernales - nearest hotel: Hostal Doña Chava
arraial do cabo - nearest hotel: No hotel found
never - nearest hotel: Hôtel de Verdun
jardim - nearest hotel: Hotel Uchôa
ushtobe - nearest hotel: No hotel found
santa maria - nearest hotel: Hotel Morotin
rio cauto - nearest hotel: No hotel found
abyek - nearest hotel: No hotel found
hami - nearest hotel: Хами
neftcala - nearest hotel: No hotel found
san ignacio - nearest hotel: Rainforest Haven Inn
bani - nearest hotel: HOTEL GLAMOURS
aksu - nearest hotel: Akesu Tianyuan International Hotel
hatillo - nearest hotel: No hotel found
flinders - nearest hotel: Nova Kiama
panaba - nearest hotel: No hotel found
manzanillo - nearest hotel: Hotel Venus
tigre - nearest hotel: Posada de 1860
san lorenzo -

,City,Country,Lat,Lng,Humidity,Hotel Name
37,pisco,PE,-13.7000,-76.2167,83,La Portada
86,coahuayana,MX,18.7333,-103.6833,72,No hotel found
101,chuy,UY,-33.6971,-53.4616,94,Alerces
129,verrettes,HT,19.0500,-72.4667,82,Kay Haiti
137,birin,NP,26.8865,88.0718,21,Sun Hotel and Lodge
185,cidreira,BR,-30.1811,-50.2056,86,Hotel Castelo
260,pedernales,DO,18.0384,-71.7440,55,Hostal Doña Chava
262,arraial do cabo,BR,-22.9661,-42.0278,94,No hotel found
279,never,FR,46.9896,3.1590,58,Hôtel de Verdun
286,jardim,BR,-21.4803,-56.1381,87,Hotel Uchôa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [164]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 680,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols= ["City", "Country", "Humidity", "Hotel Name"],
    alpha = 0.6
)

# # Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)